## Setup

In [ ]:
from specific import *

### Get unshifted data

In [ ]:
# XXX:
# data_memory.clear()

(
    endog_data,
    exog_data,
    master_mask,
    filled_datasets,
    masked_datasets,
    land_mask,
) = get_data()

In [ ]:
selection_datasets = [
    AvitabileThurnerAGB(),
    Copernicus_SWI(),
    ERA5_Temperature(),
    ESA_CCI_Landcover_PFT(),
    GFEDv4(),
    HYDE(),
    WWLLN(),
]
# These datasets will potentially be shifted.
datasets_to_shift = [
    ERA5_DryDayPeriod(),
    MOD15A2H_LAI_fPAR(),
    VODCA(),
    GlobFluo_SIF(),
]
selection_datasets += datasets_to_shift
dataset_times(selection_datasets, lat_lon=True)[2]

In [ ]:
_ = cube_plotting(master_mask.astype("float"))

In [ ]:
for pretty_name in sort_features(masked_datasets.pretty_variable_names):
    cube = masked_datasets.cubes[
        masked_datasets.pretty_variable_names.index(pretty_name)
    ]
    bounds = []
    for i in range(cube.shape[0]):
        bounds.extend(cube.coord("time").cell(i).bound)
    print(
        f"{pretty_name:<30} {ensure_datetime(min(bounds)):%Y-%m-%d} {ensure_datetime(max(bounds)):%Y-%m-%d}"
    )

In [ ]:
for pretty_name in sort_features(masked_datasets.pretty_variable_names):
    cube = masked_datasets.cubes[
        masked_datasets.pretty_variable_names.index(pretty_name)
    ]

    if any(
        name in pretty_name
        for name in ("Dry Day Period", "GFED4 BA", "popd", "lightning")
    ):
        log = True
    else:
        log = False

    cube_plotting(cube.collapsed("month_number", iris.analysis.MEAN), log=log)
    cube_plotting(cube.data.mask.astype("float"), title=f"{pretty_name} Mask")

### Get shifted data

In [ ]:
(
    endog_data,
    exog_data,
    master_mask,
    filled_datasets,
    masked_datasets,
    land_mask,
) = get_offset_data()

## Mapping

In [ ]:
with figure_saver("high_fapar_high_dry_day_period", sub_directory="map_plots"):
    mpl.rc("figure", figsize=(11, 4))
    constrained_map_plot(
        {"FAPAR": (0.36, None), "Dry Day Period": (18, None)},
        exog_data,
        master_mask,
        plot_variable="FAPAR",
        coastline_kwargs={"linewidth": 0.5},
    )

In [ ]:
with figure_saver("high_dry_day_period_18_medium_agbtree", sub_directory="map_plots"):
    mpl.rc("figure", figsize=(11, 4))
    constrained_map_plot(
        {"Dry Day Period -18 - -6 Month": (22, None), "AGB Tree": (0.9, 20)},
        exog_data,
        master_mask,
        plot_variable="AGB Tree",
        coastline_kwargs={"linewidth": 0.5},
    )

In [ ]:
with figure_saver("high_pftCrop", sub_directory="map_plots"):
    mpl.rc("figure", figsize=(11, 4))
    constrained_map_plot(
        {"pftCrop": (0.6, None)},
        exog_data,
        master_mask,
        plot_variable="pftCrop",
        coastline_kwargs={"linewidth": 0.5},
    )

## Correlation Plot

In [ ]:
X_corr = exog_data
with figure_saver("corr_plot"):
    corr_plot(
        repl_fill_names_columns(
            shorten_columns(
                X_corr[sort_features(filter_by_month(X_corr.columns, None, 9))]
            )
        ),
        fig_kwargs={"figsize": (12, 8)},
    )
    plt.gca().grid(False)

In [ ]:
with figure_saver("corr_plot_full"):
    corr_plot(
        repl_fill_names_columns(shorten_columns(X_corr[sort_features(X_corr.columns)])),
        fig_kwargs={"figsize": (7.35, 12)},
        colorbar_kwargs=dict(pad=0.015, shrink=0.15, aspect=25),
        rotation=70,
    )
    plt.gca().grid(False)

In [ ]:
from wildfires.data.datasets import *

agb = AvitabileThurnerAGB()
_ = cube_plotting(agb.cube)